# Good-Turing Smoothing Language Model

In [1]:
import sys
import re
from math import log10
from collections import Counter

In [2]:
# input file name
filename = 'VOA.UTF8.txt'
# Vocabulary Size
V = 80000
# k = 10
k = 10

In [3]:
def to_ngrams(unigrams, length):
    return zip(*[unigrams[i:] for i in range(length)])

In [4]:
# compute unigram and bigram count
unigram_counter = Counter()
bigram_counter = Counter()

for line in open(filename, 'r'):
    words = line.strip().lower().split()
    unigram_counter.update(words)
    bigram_counter.update(to_ngrams(words, 2))

In [5]:
print(unigram_counter.most_common(5))

[('the', 174276), ('.', 124709), (',', 107373), ('to', 74496), ('of', 71847)]


In [6]:
print(bigram_counter.most_common(5))

[(('of', 'the'), 17426), (('in', 'the'), 15846), (('the', 'united'), 8512), ((',', 'and'), 7457), ((',', 'the'), 7202)]


In [7]:
# compute N1, N2, N3...
unigram_Nr = Counter(unigram_counter.values())
bigram_Nr = Counter(bigram_counter.values())
# compute N0
unigram_Nr[0] = V - len(unigram_counter)
bigram_Nr[0] = V ** 2 - len(bigram_counter)
print(unigram_Nr[0], bigram_Nr[0])

43037 6399548570


In [8]:
# compute r
unigram_r = [(i+1) * unigram_Nr[i+1] / unigram_Nr[i] for i in range(k)]
bigram_r = [(i+1) * bigram_Nr[i+1] / bigram_Nr[i] for i in range(k)]
print(unigram_r)
print(bigram_r)

[0.2869623812068685, 0.7876923076923077, 1.742393092105263, 2.816283185840708, 3.5671191553544497, 4.558139534883721, 5.681818181818182, 6.153142857142857, 7.863298662704309, 9.200680272108844]
[3.8979936986399026e-05, 0.5488146111106657, 1.5000438263308595, 2.415052443976977, 3.3005081054923786, 4.260391466901254, 5.099318604170969, 6.272108843537415, 7.145336225596529, 8.039617486338798]


In [9]:
# compute normalize factor
# compute N
unigram_N = sum(unigram_counter.values())
bigram_N = sum(bigram_counter.values())
print(unigram_N, bigram_N)

2914867 2789569


In [22]:
# compute new probability sum
unigram_N_ = unigram_N + k * unigram_Nr[k]
bigram_N_ = bigram_N + k * bigram_Nr[k]
print(unigram_N_, bigram_N_)

2920277 2824879


In [21]:
# normalize factor: N/N’
unigram_norm_factor = unigram_N / unigram_N_
bigram_norm_factor = bigram_N / bigram_N_
print(unigram_norm_factor, bigram_norm_factor)

0.9981474360137754 0.987500349572495


In [11]:
# Estimating P(w) and P(w’|w)
def prob_1word(unigram):
    count = unigram_counter[unigram]
    r = unigram_r[count] if count < k else count
    return log10(r / unigram_N_)
def prob_2words(text_front, text_rear):
    count = bigram_counter[text_front, text_rear]
    r = bigram_r[count] if count < k else count
    return log10(r / bigram_N_)
def prob_word_by_word(text_front, text_rear):
    return prob_2words(text_front, text_rear) - prob_1word(text_front)
def prob_words(words):
    return prob_1word(words[0]) + sum(prob_word_by_word(words[i-1], words[i]) for i in range(1, len(words)))
def prob_text(text):
    return prob_words(text.lower().split())

In [12]:
prob_1word('this')

-2.7286292930534612

In [13]:
prob_2words('this', 'is')

-3.6509704908731844

In [14]:
prob_word_by_word('this', 'is')

-0.9223411978197231

In [15]:
prob_text('this is an book .')

-15.554022361802009

In [16]:
prob_text('this is a book .')

-9.630625298586526

In [17]:
prob_text('she can speak english .')

-11.738302830233998

In [18]:
prob_text('she can say english .')

-17.595264125895014

In [19]:
# you can use log to speed up the calculation
unicount_log = {k: log10(v) for k, v in unigram_counter.items()}
bicount_log = {k: log10(v) for k, v in bigram_counter.items()}
unigram_r_log = [log10(r) for r in unigram_r]
bigram_r_log = [log10(r) for r in bigram_r]
unigram_N_log = log10(unigram_N_)
bigram_N_log = log10(bigram_N_)


def prob_1word(unigram):
    count = unigram_counter[unigram]
    r = unigram_r_log[count] if count < k else unicount_log[unigram]
    return r - unigram_N_log
def prob_2words(text_front, text_rear):
    count = bigram_counter[text_front, text_rear]
    r = bigram_r_log[count] if count < k else bicount_log[text_front, text_rear]
    return r - bigram_N_log